In [32]:
import pandas as pd

#Read the csv File
#Try Common Encodings: As a last resort, you can try common encodings like
#'utf-8', 'latin1', 'ISO-8859-1', etc., and see if any of them successfully decode the file without errors.

# Adjust display options
pd.set_option('display.max_columns', None)  # Show all columns

df = pd.read_csv('Bundesligadaten.txt', delimiter='\t', encoding='ISO-8859-1', header=None)

df.columns = ['date', 'time', 'localTeam', 'awayTeam', 'score', 'score-HT']


In [33]:
#df['date'].isnull().any()

In [34]:
for column in ('date', 'time'):
     # Check if the column contains NaN values
    if df[column].isnull().any():
        # Fill NaN values with corresponding values from the last row using "forward fill"
        df[column] = df[column].fillna(method='ffill')

In [35]:

# Split the date Value to create a Day of the Week       
df['DoW'] = df['date'].str.split(',').str[0]
df['date'] = df['date'].str.split(',').str[1]

# Split the score
split_values = df['score'].str.split(':')
df['localTeam-Score'] = split_values.str[0].str[-1]
df['awayTeam-Score'] = split_values.str[1].str[0]

# Split the score to half time
split_values = df['score-HT'].str.split(':')
df['localTeam-SHT'] = split_values.str[0].str[-1]
df['awayTeam-SHT'] = split_values.str[1].str[0]


# Reorder columns
df = df[['DoW', 'date', 'time', 'localTeam', 'awayTeam', 'score', 'localTeam-Score', 'awayTeam-Score', 'score-HT', 'localTeam-SHT', 'awayTeam-SHT']]

In [36]:
df.head(1)

,DoW,date,time,localTeam,awayTeam,score,localTeam-Score,awayTeam-Score,score-HT,localTeam-SHT,awayTeam-SHT
0,Fr,05.08.2022,20:30,Eintracht Frankfurt,- FC Bayern München,1:6,1,6,(0:5),0,5


In [37]:
# Triming Columns
df['score-HT'] = df['score-HT'].str.removeprefix('(').str.removesuffix(')')
df['awayTeam'] = df['awayTeam'].str.removeprefix('-')

In [38]:
df.head(1)

,DoW,date,time,localTeam,awayTeam,score,localTeam-Score,awayTeam-Score,score-HT,localTeam-SHT,awayTeam-SHT
0,Fr,05.08.2022,20:30,Eintracht Frankfurt,FC Bayern München,1:6,1,6,0:5,0,5


In [78]:
import numpy as np    
# Generate 'NewField' based on comparison of 'Field1' and 'Field2'
# df['points-Local'] = df.apply(lambda row: 3 if row['localTeam-Score'] > row['awayTeam-Score'] else (1 if ((row['localTeam-Score'] == row['awayTeam-Score']) and row['score' != np.NaN]) else (0 if row['localTeam-Score'] < row['awayTeam-Score'] else np.NaN )), axis=1)
df['points-Local'] = df.apply(lambda row: 3 if row['localTeam-Score'] > row['awayTeam-Score'] else (1 if ((row['localTeam-Score'] == row['awayTeam-Score']) and row['awayTeam-Score'] != np.NaN) else (0 if row['localTeam-Score'] < row['awayTeam-Score'] else 'TBD')), axis=1)
# df['points-Local'] = df.apply(lambda row: type(row['points-Local']) if row['points-Local'] >= 0 else row['localTeam-Score'], axis=1)
# df['points-Local'] = df['points-Local'].to_int()

#######
df['points-Away'] = df.apply(lambda row: 3 if row['awayTeam-Score'] > row['localTeam-Score'] else (1 if ((row['localTeam-Score'] == row['awayTeam-Score']) and row['awayTeam-Score'] != np.NaN) else (0 if row['awayTeam-Score'] < row['localTeam-Score'] else 'TBD')), axis=1)
#######
# Reorder columns
df = df[['DoW', 'date', 'time', 'localTeam', 'points-Local', 'awayTeam', 'points-Away', 'score', 'localTeam-Score', 'awayTeam-Score', 'score-HT', 'localTeam-SHT', 'awayTeam-SHT']]

In [79]:
df['points-Local']

0        0
1        3
2        3
3        0
4        0
      ... 
301    TBD
302    TBD
303    TBD
304    TBD
305    TBD
Name: points-Local, Length: 306, dtype: object

In [80]:
blTable = df.groupby('localTeam')['points-Local'].sum()

blTable.sort_values(ascending=False)


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [42]:
blTable_away = df.groupby('awayTeam')['points-Away'].sum()

blTable_away.sort_values(ascending=False)

awayTeam
 1. FC Köln                0
 1. FC Union Berlin        0
 VfL Wolfsburg             0
 VfL Bochum                0
 VfB Stuttgart             0
 TSG Hoffenheim            0
 SC Freiburg               0
 Rasenballsport Leipzig    0
 Hertha BSC                0
 FSV Mainz 05              0
 FC Schalke 04             0
 FC Bayern München         0
 FC Augsburg               0
 Eintracht Frankfurt       0
 Borussia Dortmund         0
 Bor. Mönchengladbach      0
 Bayer 04 Leverkusen       0
 Werder Bremen             0
Name: points-Away, dtype: int64

In [43]:
# Filter rows where 'localTeam' is 'Team A'
df[df['localTeam'] == 'Eintracht Frankfurt']
# df[df['awayTeam'] == 'Eintracht Frankfurt']

,DoW,date,time,localTeam,points-Local,awayTeam,points-Away,score,localTeam-Score,awayTeam-Score,score-HT,localTeam-SHT,awayTeam-SHT
0,Fr,05.08.2022,20:30,Eintracht Frankfurt,Loser,FC Bayern München,0,1:6,1,6,0:5,0,5
25,So,21.08.2022,15:30,Eintracht Frankfurt,draw,1. FC Köln,0,1:1,1,1,0:0,0,0
42,Sa,03.09.2022,18:30,Eintracht Frankfurt,Winner,Rasenballsport Leipzig,0,4:0,4,0,2:0,2,0
46,Sa,10.09.2022,15:30,Eintracht Frankfurt,Loser,VfL Wolfsburg,0,0:1,0,1,0:0,0,0
65,Sa,01.10.2022,15:30,Eintracht Frankfurt,Winner,1. FC Union Berlin,0,2:0,2,0,2:0,2,0
82,Sa,15.10.2022,15:30,Eintracht Frankfurt,Winner,Bayer 04 Leverkusen,0,5:1,5,1,1:0,1,0
104,Sa,29.10.2022,18:30,Eintracht Frankfurt,Loser,Borussia Dortmund,0,1:2,1,2,1:1,1,1
123,Mi,09.11.2022,20:30,Eintracht Frankfurt,Winner,TSG Hoffenheim,0,4:2,4,2,3:1,3,1
137,Sa,21.01.2023,15:30,Eintracht Frankfurt,draw,FC Schalke 04,0,:,NaN,NaN,:,(,)
166,Sa,04.02.2023,15:30,Eintracht Frankfurt,draw,Hertha BSC,0,:,NaN,NaN,:,(,)
